In [1]:
path = "/home/amit/Downloads/review-data/train_data.txt"

In [2]:
ratemap = {'1.0':0,'2.0':0,'3.0':0,'4.0':1,'5.0':1,'1':1,'0':0}

In [3]:
ratemap = {'1.0':0,'2.0':0,'3.0':0,'4.0':1,'5.0':1,'1':1,'0':0}
data = open(path)
X_train, y_train = [], []
for line in data:
    text = line.partition(",")
    if text[0] in ratemap.keys():
        X_train.append(text[2])
        y_train.append(ratemap[text[0]])

In [4]:
print len(X_train)

25166


In [5]:
print X_train[0]
print y_train[0]

I picked up a dozen in Home Depot for less than $5. Same concept, just a lot cheaper!Two of them are holding a huge heavy shelf in my kitchen. The hole they make depends upon the weight of what they're holding. I'm sure it's pinhole sized if you're hanging something that weighs a few ounces (in which case, you don't need this - use a pushpin!)The holes from the shelf are bigger than pin-sized, but it IS a heavy shelf. The hole is still WAY smaller than the hole I'd make using any kind of a hook that would hold such a heavy shelf.It also takes a little practice to figure out where to hang the hooks (the drop of the hook is below where you might want the thing to actually hang.) This is a small detail, but if you need two of these things and you don't do it right, your object is gonna be crooked. But on the bright side, you won't have a huge hole to patch when you redo it!Recommended product (but look around for the knockoffs.

1


In [6]:
import re
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
def tokenizer(text):
    text = re.sub('<[^>]*>','',text)
    smileys = re.findall('((?::|;|=)(?:-?)(?:[D|d|)|(|P|p|/|x|X]))',text)
    text = re.sub('[\W]+',' ',text.lower())
    text += ' '.join(smileys).replace('-','')
    tokenized = [w for w in text.split() if w not in stop]
    return " ".join(tokenized)

In [7]:
import numpy as np

X = [tokenizer(text) for text in X_train]

X_train = np.asarray(X)
y_train = np.asarray(y_train)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

def create_nb_ngram_model():
    tfidf_ngrams = TfidfVectorizer(ngram_range=(1,3),
                                  analyzer="word",binary=False,
                                  max_features=8000)
    clf = MultinomialNB()
    pipeline = Pipeline([('vect',tfidf_ngrams),
                        ('clf',clf)])
    return pipeline

In [12]:
from sklearn.linear_model import LogisticRegression

def create_lr_ngram_model():
    tfidf_ngrams = TfidfVectorizer(ngram_range=(1,3),
                                  analyzer="word",binary=False,
                                  max_features=8000)
    clf = LogisticRegression()
    pipeline = Pipeline([('vect',tfidf_ngrams),
                        ('clf',clf)])
    return pipeline

In [13]:
from sklearn.metrics import precision_recall_curve, auc
from sklearn.cross_validation import ShuffleSplit

def train_model(clf_factory, X, y):
    cv = ShuffleSplit(n=len(X), n_iter=10, test_size=0.3,
                    random_state=0)
    
    scores = []
    pr_scores = []
    
    for train, test in cv:
        X_t, y_t = X[train], y[train]
        X_test, y_test = X[test], y[test]
        
        clf = clf_factory
        clf.fit(X_t,y_t)
        
        train_score = clf.score(X_t,y_t)
        test_score = clf.score(X_test,y_test)
        
        scores.append(test_score)
        proba = clf.predict_proba(X_test)
        
        precision, recall, pr_thresholds = precision_recall_curve(y_test, proba[:,1])
        
        pr_scores.append(auc(recall,precision))
    
    summary = (np.mean(scores), np.std(scores),
              np.mean(pr_scores), np.std(pr_scores))
    
    print summary

In [10]:
pipeline = create_nb_ngram_model()
train_model(pipeline,X_train,y_train)

(0.84532450331125841, 0.0031235820869899098, 0.92704051603722115, 0.0022478866102008887)


/usr/local/lib/python2.7/dist-packages/numpy/core/fromnumeric.py:2641: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


In [14]:
pipeline = create_lr_ngram_model()
train_model(pipeline,X_train,y_train)

(0.86452980132450319, 0.0032220023379616793, 0.9400682185897844, 0.0022848491508089138)
